# Регрессия с Statsmodels. Диагностика

In [14]:
import numpy as np
import pandas as pd

import statsmodels.formula.api as smf
from statsmodels.stats import diagnostic # диагностика модели
from statsmodels.stats.stattools import durbin_watson

from statsmodels.iolib.summary2 import summary_col

## Пример: гетероскедастичность для sleep equation

In [2]:
df = pd.read_csv('sleep75.csv')

На датасете/датафрейме `df` подгоним регрессия `sleep` на `totwrk, age, age^2, male, smsa, marr`.

In [3]:
# инициализация/спецификация модели через формулу
# создаём объект класса OLS
mod = smf.ols(formula='sleep~totwrk+age+I(age**2)+male+smsa+marr', data=df)

# подгонка специфицированной модели на данных с неробастной ковариационной матрицей для коэффициентов
# создаём объект класса RegressionResults
res = mod.fit()

In [4]:
# тест на гетероскедастичность
diagnostic.het_breuschpagan(res.resid, exog_het=mod.exog)

(8.421166040057434,
 0.20884214179658228,
 1.4063870574992607,
 0.20946231383258354)

## Пример: серийная корреляция

In [5]:
df= pd.read_csv('Mishkin.csv')

На датасете/датафрейме `df` подгоним регрессия `pai3` на `tb1, tb3`.

In [6]:
# инициализация/спецификация модели через формулу
# создаём объект класса OLS
mod = smf.ols(formula='pai3~tb1+tb3', data=df)

# подгонка специфицированной модели на данных с неробастной ковариационной матрицей для коэффициентов
# создаём объект класса RegressionResults
res = mod.fit()

In [7]:
diagnostic.acorr_breusch_godfrey(res=res, nlags=2)

(369.14827506022044,
 6.9258099284231825e-81,
 736.1654575178613,
 8.400323459468088e-148)

In [8]:
# Статистика Durbin-Watson
durbin_watson(resids=res.resid)

0.2936716432769036

In [12]:
res_hac = mod.fit(cov_type='HAC', cov_kwds={'maxlags': 3})

In [16]:
summary_col(results=[res, res_hac], model_names=['non-robust', 'robust'], stars=True, info_dict={'F': lambda x: x.fvalue, 'Nobs': lambda x: x.nobs})

,non-robust,robust
Intercept,0.9726***,0.9726**
,(0.2404),(0.4803)
tb1,0.6081*,0.6081
,(0.3166),(0.4512)
tb3,0.0012,0.0012
,(0.3016),(0.4472)
R-squared,0.3255,0.3255
R-squared Adj.,0.3227,0.3227
F,117.7228,31.3701
Nobs,491.0000,491.0000
